In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
from common.layers import softmax, Convolution, MaxPooling, ReLU, Affine, SoftmaxWithLoss
from common.optimizer import RMSProp, Adam
from collections import OrderedDict

In [2]:
class cnn_homebrew:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':1, 'stride':1},
                 pool_param={'pool_size':2, 'pad':0, 'stride':2},
                 hidden_size=100, output_size=15, weight_init_std=0.01,
                batch_size=500):
        """
        input_size : tuple, 入力の配列形状(チャンネル数、画像の高さ、画像の幅)
        conv_param : dict, 畳み込みの条件
        pool_param : dict, プーリングの条件
        hidden_size : int, 隠れ層のノード数
        output_size : int, 出力層のノード数
        weight_init_std ： float, 重みWを初期化する際に用いる標準偏差
        """
                
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        
        pool_size = pool_param['pool_size']
        pool_pad = pool_param['pad']
        pool_stride = pool_param['stride']
        
        input_size = input_dim[1]
        conv_output_size = (input_size + 2*filter_pad - filter_size) // filter_stride + 1 # 畳み込み後のサイズ(H,W共通)
        pool_output_size = (conv_output_size + 2*pool_pad - pool_size) // pool_stride + 1 # プーリング後のサイズ(H,W共通)
        pool_output_pixel = filter_num * pool_output_size * pool_output_size # プーリング後のピクセル総数
        
        # 重みの初期化
        self.params = {}
        std = weight_init_std
        self.batch_size = batch_size
        self.params['W1'] = std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size) # W1は畳み込みフィルターの重みになる
        self.params['b1'] = np.zeros(filter_num) #b1は畳み込みフィルターのバイアスになる
        self.params['W2'] = std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size) # W1は畳み込みフィルターの重みになる
        self.params['b2'] = np.zeros(filter_num) 
        self.params['W3'] = std *  np.random.randn(pool_output_pixel, hidden_size)
        self.params['b3'] = np.zeros(hidden_size)
        self.params['W4'] = std *  np.random.randn(hidden_size, output_size)
        self.params['b4'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad']) # W1が畳み込みフィルターの重み, b1が畳み込みフィルターのバイアスになる
        self.layers['ReLU1'] = ReLU()
        self.layers['Pool1'] = MaxPooling(pool_h=pool_size, pool_w=pool_size, stride=pool_stride, pad=pool_pad)
        
        self.layers['Conv2'] = Convolution(self.params['W2'], self.params['b2'],
                                           conv_param['stride'], conv_param['pad']) # W1が畳み込みフィルターの重み, b1が畳み込みフィルターのバイアスになる
        self.layers['ReLU2'] = ReLU()
        self.layers['Pool2'] = MaxPooling(pool_h=pool_size, pool_w=pool_size, stride=pool_stride, pad=pool_pad)
        
        self.layers['Affine1'] = Affine(self.params['W3'], self.params['b3'])
        self.layers['ReLU3'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W4'], self.params['b4'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """
        損失関数
        x : 入力データ
        t : 教師データ
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / self.batch_size)):
            tx = x[i*self.batch_size:(i+1)*self.batch_size]
            tt = t[i*self.batch_size:(i+1)*self.batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def gradient(self, x, t):
        """勾配を求める（誤差逆伝播法）
        Parameters
        ----------
        x : 入力データ
        t : 教師データ
        Returns
        -------
        各層の勾配を持ったディクショナリ変数
            grads['W1']、grads['W2']、...は各層の重み
            grads['b1']、grads['b2']、...は各層のバイアス
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Conv2'].dW, self.layers['Conv2'].db
        grads['W3'], grads['b3'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W4'], grads['b4'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads


In [3]:
#データを読む
train_data = np.load("../1_data/train_data.npy")
train_label = np.load("../1_data/train_label.npy")
print("train_data.shape=", train_data.shape)
print("train_label.shape=", train_label.shape)

# 正規化
train_data = (train_data - train_data.min()) / train_data.max()
train_data = train_data.astype('float32')

train_data = train_data.reshape(-1, 28*28)
#データ分割
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, 
                                                    test_size=0.3, random_state=1234,
                                                    shuffle=True)
print('X_train.shape=', X_train.shape)
print('y_train.shape=',y_train.shape)
print('X_test.shape=', X_test.shape)
print('y_test.shape=',y_test.shape)

# train = X_train/255
# test = X_test/255
# train = train.reshape(-1, 1, 28*28)
# test = test.reshape(-1, 1, 28*28)
# print(train.shape)
# 正規化
# train = (train - train.min()) / train.max()
# train = train.astype('float32')

# from sklearn.preprocessing import LabelBinarizer
# lb = LabelBinarizer()
# train_labels = lb.fit_transform(y_train)
# test_labels = lb.fit_transform(y_test)

train_data.shape= (3000, 1, 28, 28)
train_label.shape= (3000, 15)
X_train.shape= (2100, 784)
y_train.shape= (2100, 15)
X_test.shape= (900, 784)
y_test.shape= (900, 15)


In [4]:
X_train = X_train.reshape(-1,1,28,28)
X_test = X_test.reshape(-1, 1, 28, 28)

In [5]:
x = X_train[:2000,:] #15000, epoch=1,0.76. 20000,18000はできない．
t = y_train[:2000,:]

x = x.reshape(-1,1,28,28) # 配列形式の変形

epochs = 5 #27
batch_size = 200 #500

#optimizer = RMSProp(lr=0.01, rho=0.9)
optimizer = Adam(lr=0.01, beta1=0.9, beta2=0.999)

# 繰り返し回数
xsize = x.shape[0]

iter_num = np.ceil(xsize / batch_size).astype('int')


# CNNのオブジェクト生成
model = cnn_homebrew(input_dim=(1, 28, 28), 
                     conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                     pool_param={'pool_size':2, 'pad':0, 'stride':2},
                     hidden_size=500, output_size=15, weight_init_std=0.01,
                    batch_size=batch_size)

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

for epoch in range(epochs):
    print("epoch=%s"%epoch)

    # シャッフル
    idx = np.arange(xsize)
    np.random.shuffle(idx)

    for it in range(iter_num):
        """
        ランダムなミニバッチを順番に取り出す
        """
        #print("it=", it)
        mask = idx[batch_size*it : batch_size*(it+1)]

        # ミニバッチの生成
        x_train = x[mask]
        t_train = t[mask]
        
        # 勾配の計算 (誤差逆伝播法を用いる) 
        grads = model.gradient(x_train, t_train)
        # 更新
        optimizer.update(model.params, grads)

    ## 学習経過の記録

    # 訓練データにおけるloss
#     print("calculating train_loss") 
    train_loss.append(model.loss(x,  t))

#     print("calculating test_loss")
    # テストデータにおけるloss
    print("tl%s"%epoch)
    test_loss.append(model.loss(X_test, y_test))
    print(f"test_loss,{test_loss[epoch]}")
#     print("calculating train_accuracy")
    # 訓練データにて精度を確認
    train_accuracy.append(model.accuracy(x, t))
          
#     print("calculating test_accuracy")
    # テストデータにて精度を算出
    print("ta%s"%epoch)
    test_accuracy.append(model.accuracy(X_test, y_test))
    print(f"test_acc,{test_accuracy[epoch]}")
    if test_accuracy[epoch]>=0.98 and test_loss[epoch]<0.001:
        break


epoch=0


ValueError: shapes (12800,750) and (25,30) not aligned: 750 (dim 1) != 25 (dim 0)

In [ ]:
# lossとaccuracyのグラフ化
df_log = pd.DataFrame({"train_loss":train_loss,
             "test_loss":test_loss,
             "train_accuracy":train_accuracy,
             "test_accuracy":test_accuracy})

df_log.plot(style=['r-', 'r--', 'b-', 'b--'])
plt.ylim([0,4])
plt.ylabel("Accuracy or loss")
plt.xlabel("epochs")
plt.show()

In [ ]:
あ

In [ ]:
test_loss,0.030907047067291988
test_acc,0.9869907407407408
epoch=12

In [ ]:
with open('katakana_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('model_params.pickle', 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
あ

In [ ]:
def init_network():
    with open('katakana_model.pickle', 'rb') as f:
        network = pickle.load(f)
    return network

notwork = init_network()
print(network.keys())

In [ ]:
def init_network():
    with open("sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    return network

network = init_network()
print(network.keys())

In [ ]:
model = TwoLayerNet(input_size=28*28, hidden_size=10, output_size=15)
    with open("params.pickle", "rb") as f:
        params = pickle.load(f)
        for i, key in enumerate(['Affine1', 'Affine2']):
            model.layers[key].W = params['W' + str(i+1)]
            model.layers[key].b = params['b' + str(i+1)]
            model.params[key].W = params['W' + str(i+1)]
            model.params[key].b = params['b' + str(i+1)]


In [ ]:
a = [1,2,3]
for i in range(3):
    print(i,a[i])